In [1]:
# For data manipulation

import pandas as pd

import numpy as np 

# machine learning classification

from sklearn.svm import SVC

from sklearn.metrics import scorer

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# To plot
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn

# To fetch data

from pandas_datareader import data as pdr


In [2]:
import glob
import pandas as pd
pattern='*.csv'
hong = glob.glob(pattern)
len(hong)

62

In [3]:
hkong = pd.read_csv(hong[0])
hkong = hkong.drop('Adj Close', axis=1)
hkong.columns = ['Date', 'Open' + '_'+str(0), 'High'+'_'+str(0), 'Low' +'_'+str(0), 'Close'+ '_'+str(0), 'Volume'+ '_'+ str(0)]

for date in hkong['Date']:
    hkong['Date'] = pd.to_datetime(hkong['Date'])
hkong =hkong.sort_values(by='Date')

hkong['Volume'+ '_'+ str(0)] = hkong['Volume'+ '_'+ str(0)].fillna(0)
for col in hkong:
    hkong[col] = hkong[col].fillna(method='ffill')
    
#hkong['Target'] = np.where(hkong['Close'].shift(-1) > hkong['Close'],int(1),int(-1))
hkong.shape

(1234, 6)

In [4]:
hkong['Open' + '_'+str(0)] = ((hkong['Open' + '_'+str(0)] - hkong['Open' + '_'+str(0)].shift(1))/hkong['Open' + '_'+str(0)].shift(1)) *100

hkong['High' + '_'+str(0)] = ((hkong['High' + '_'+str(0)] - hkong['High' + '_'+str(0)].shift(1))/hkong['High' + '_'+str(0)].shift(1)) *100

hkong['Low' + '_'+str(0)] = ((hkong['Low' + '_'+str(0)] - hkong['Low' + '_'+str(0)].shift(1))/hkong['Low' + '_'+str(0)].shift(1)) *100

hkong['Close' + '_'+str(0)] = ((hkong['Close' + '_'+str(0)] - hkong['Close' + '_'+str(0)].shift(1))/hkong['Close' + '_'+str(0)].shift(1)) *100

hkong = hkong.drop(0, axis=0)


print hkong.shape
hkong.head()

#hkong['Open' + '_'+str(0)] = hkong['Open' + '_'+str(0)].drop(hkong['Open' + '_'+str(0)].index[0])


(1233, 6)


,Date,Open_0,High_0,Low_0,Close_0,Volume_0
1,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412
2,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548
3,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634
4,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274
5,2013-03-18,-2.843602,-2.351097,-1.620746,0.646200,2427267


In [5]:
#run only once
for i in range(1,len(hong)):
    df = pd.read_csv(hong[i]) 
    
    if 'Adj Close' in df:
        df = df.drop('Adj Close', axis=1) 
    
    df.columns = ['Date', 'Open' + '_'+str(i), 'High'+'_'+str(i), 'Low' +'_'+str(i), 'Close'+'_'+str(i), 'Volume'+'_'+str(i)]
    
    #formatting date
    for date in df['Date']:
        df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date')
    
    #imputing missing values
    df['Volume'+ '_'+ str(i)] = df['Volume'+ '_'+ str(i)].fillna(0)
    for col in df:
        df[col] = df[col].fillna(method='ffill')
    df['Open' + '_'+str(i)] = ((df['Open' + '_'+str(i)] - df['Open' + '_'+str(i)].shift(1))/df['Open' + '_'+str(i)].shift(1)) *100

    df['High' + '_'+str(i)] = ((df['High' + '_'+str(i)] - df['High' + '_'+str(i)].shift(1))/df['High' + '_'+str(i)].shift(1)) *100

    df['Low' + '_'+str(i)] = ((df['Low' + '_'+str(i)] - df['Low' + '_'+str(i)].shift(1))/df['Low' + '_'+str(i)].shift(1)) *100

    df['Close' + '_'+str(i)] = ((df['Close' + '_'+str(i)] - df['Close' + '_'+str(i)].shift(1))/df['Close' + '_'+str(i)].shift(1)) *100

    df = df.drop(0, axis=0)
    
    hkong = pd.merge(hkong, df, how='outer', on='Date')

hkong.shape


(1555, 311)

In [17]:
#for i in range(len(hong)):
   # print hkong.loc[:, 'Volume'+'_'+str(i)].head()

In [6]:
#hkong.to_csv('E:/hkongpercent-1_missing.csv')  ---->not needed

In [6]:

for date in hkong['Date']:
    hkong['Date'] = pd.to_datetime(hkong['Date'])
hkong =hkong.sort_values(by='Date')

In [7]:
hkong.head()

,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Open_60,High_60,Low_60,Close_60,Volume_60,Open_61,High_61,Low_61,Close_61,Volume_61
1233,2013-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,-0.238700,0.531070,-1.382659,-1.038177,117600.0,0.213418,-0.321168,-0.190709,-0.547478,2738000.0
1,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,-1.168739,-2.012475,-0.511300,-0.989981,91000.0,-0.250032,-0.194338,-0.302631,0.010004,2224400.0
2,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,-1.156576,-0.442113,0.002709,0.278674,80200.0,-0.205514,-0.557313,-0.202575,-0.547182,2150200.0
3,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0.600239,1.575284,-0.146681,0.357832,116400.0,-0.159327,-0.034864,-0.268680,-0.305210,2530400.0


In [8]:
for i in range(0,len(hong)):
    hkong['Volume'+ '_'+ str(i)] = hkong['Volume'+ '_'+ str(i)].fillna(0)
for col in hkong.columns:
    hkong[col] = hkong[col].fillna(method='ffill')
hkong.head()

,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Open_60,High_60,Low_60,Close_60,Volume_60,Open_61,High_61,Low_61,Close_61,Volume_61
1233,2013-03-11,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
0,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,-0.238700,0.531070,-1.382659,-1.038177,117600.0,0.213418,-0.321168,-0.190709,-0.547478,2738000.0
1,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,-1.168739,-2.012475,-0.511300,-0.989981,91000.0,-0.250032,-0.194338,-0.302631,0.010004,2224400.0
2,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,-1.156576,-0.442113,0.002709,0.278674,80200.0,-0.205514,-0.557313,-0.202575,-0.547182,2150200.0
3,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0.600239,1.575284,-0.146681,0.357832,116400.0,-0.159327,-0.034864,-0.268680,-0.305210,2530400.0


In [9]:

hkong = hkong.fillna(0)

In [10]:
hkong.head()

,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Open_60,High_60,Low_60,Close_60,Volume_60,Open_61,High_61,Low_61,Close_61,Volume_61
1233,2013-03-11,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,-0.238700,0.531070,-1.382659,-1.038177,117600.0,0.213418,-0.321168,-0.190709,-0.547478,2738000.0
1,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,-1.168739,-2.012475,-0.511300,-0.989981,91000.0,-0.250032,-0.194338,-0.302631,0.010004,2224400.0
2,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,-1.156576,-0.442113,0.002709,0.278674,80200.0,-0.205514,-0.557313,-0.202575,-0.547182,2150200.0
3,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0.600239,1.575284,-0.146681,0.357832,116400.0,-0.159327,-0.034864,-0.268680,-0.305210,2530400.0


In [11]:
hkong.isnull().sum()[hkong.isnull().sum() !=0]

Series([], dtype: int64)

In [12]:
for i in range(30):
    hkong['Target'+'_'+str(i)] = np.where(hkong['Close'+'_'+str(i)]>0.1,1,0)
    
hkong.shape

(1555, 341)

In [13]:
hkong.head()

,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Target_20,Target_21,Target_22,Target_23,Target_24,Target_25,Target_26,Target_27,Target_28,Target_29
1233,2013-03-11,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
0,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,0,0,0,1,0,0,0,0,0,0
1,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,0,0,0,0,0,0,0,0,0,0
2,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,0,1,0,0,0,0,1,1,0,0
3,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0,0,0,0,0,0,0,0,0,0


In [14]:
hkong.index = hkong.Date
hkong = hkong.drop('Date', axis=1)
hkong.shape

(1555, 340)

In [15]:
hkong.head()

,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,Volume_1,...,Target_20,Target_21,Target_22,Target_23,Target_24,Target_25,Target_26,Target_27,Target_28,Target_29
Date,,,,,,,,,,,,,,,,,,,,,
2013-03-11,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,23618156.0,...,0,0,0,1,0,0,0,0,0,0
2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,31644955.0,...,0,0,0,0,0,0,0,0,0,0
2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,21286152.0,...,0,1,0,0,0,0,1,1,0,0
2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,44707880.0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
#hkong.to_csv('E:/hkongpercent-1_ord_currency.csv')

In [29]:
#whenever open fresh, start from here
#hkong = pd.read_csv('E:/hkongpercent-1_ord_currency.csv')   
##print hkong.shape
#hkong.head()

(1555, 341)


,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Target_20,Target_21,Target_22,Target_23,Target_24,Target_25,Target_26,Target_27,Target_28,Target_29
0,2013-03-11,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,0,0,0,1,0,0,0,0,0,0
2,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,0,0,0,0,0,0,0,0,0,0
3,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,0,1,0,0,0,0,1,1,0,0
4,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0,0,0,0,0,0,0,0,0,0


In [30]:
#hkong.index = hkong.Date
#hkong = hkong.drop('Date', axis=1)
#hkong.shape

(1555, 340)

In [16]:
hkong.head()

,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,Volume_1,...,Target_20,Target_21,Target_22,Target_23,Target_24,Target_25,Target_26,Target_27,Target_28,Target_29
Date,,,,,,,,,,,,,,,,,,,,,
2013-03-11,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,23618156.0,...,0,0,0,1,0,0,0,0,0,0
2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,31644955.0,...,0,0,0,0,0,0,0,0,0,0
2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,21286152.0,...,0,1,0,0,0,0,1,1,0,0
2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,44707880.0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
target=hkong.iloc[:, 310:].columns
target

Index([u'Target_0', u'Target_1', u'Target_2', u'Target_3', u'Target_4',
       u'Target_5', u'Target_6', u'Target_7', u'Target_8', u'Target_9',
       u'Target_10', u'Target_11', u'Target_12', u'Target_13', u'Target_14',
       u'Target_15', u'Target_16', u'Target_17', u'Target_18', u'Target_19',
       u'Target_20', u'Target_21', u'Target_22', u'Target_23', u'Target_24',
       u'Target_25', u'Target_26', u'Target_27', u'Target_28', u'Target_29'],
      dtype='object')

In [17]:
X = hkong.iloc[:, :310]

sc = StandardScaler()
X = sc.fit_transform(X)
X.shape

(1555, 310)

In [18]:
scores = []
test_acc =[]
for col in target:
    y = hkong[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier = LogisticRegression()

    cv_score = np.mean(cross_val_score(classifier, X_test, y_test, cv=10, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(col, cv_score))

    classifier.fit(X_train, y_train)
    accuracy_train = accuracy_score(y_train, classifier.predict(X_train))

    accuracy_test = accuracy_score(y_test, classifier.predict(X_test))
    test_acc.append(accuracy_test*100)
    #print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))

    #print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))
print ('Mean Test Accuracy is {: .2f}%'.format(np.mean(test_acc)))
test_acc = map(lambda x:round(x,2), test_acc)
test_acc_LR_1 =test_acc
print test_acc_LR_1

CV score for class Target_0 is 0.931071428571
CV score for class Target_1 is 0.942797272107
CV score for class Target_2 is 0.911200378072
CV score for class Target_3 is 0.927365612648
CV score for class Target_4 is 0.927060606061
CV score for class Target_5 is 0.94275528565
CV score for class Target_6 is 0.943514790765
CV score for class Target_7 is 0.924614367114
CV score for class Target_8 is 0.915505599473
CV score for class Target_9 is 0.916813186813
CV score for class Target_10 is 0.92292022792
CV score for class Target_11 is 0.920619235837
CV score for class Target_12 is 0.957355401845
CV score for class Target_13 is 0.915455204216
CV score for class Target_14 is 0.937554578755
CV score for class Target_15 is 0.910029644269
CV score for class Target_16 is 0.915183150183
CV score for class Target_17 is 0.940524574669
CV score for class Target_18 is 0.914717948718
CV score for class Target_19 is 0.90910194812
CV score for class Target_20 is 0.939721343874
CV score for class Target_

In [19]:
scores = []
test_acc=[]
for col in target:
    y = hkong[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier = SVC()

    cv_score = np.mean(cross_val_score(classifier, X_test, y_test, cv=10, scoring='roc_auc'))
    scores.append(cv_score)
    #print('CV score for class {} is {}'.format(col, cv_score))

    classifier.fit(X_train, y_train)
    accuracy_train = accuracy_score(y_train, classifier.predict(X_train))

    accuracy_test = accuracy_score(y_test, classifier.predict(X_test))
    #print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))

    #print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))
    test_acc.append(accuracy_test*100)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

print ('Mean Test Accuracy is {: .2f}%'.format(np.mean(test_acc)))
test_acc = map(lambda x:round(x,2), test_acc)
test_acc_SVC_1 =test_acc
print test_acc_SVC_1

Total CV score is 0.87565127744
Mean Test Accuracy is  85.56%
[86.08, 84.15, 80.3, 85.22, 85.22, 85.01, 86.94, 84.58, 83.51, 81.58, 87.79, 86.94, 87.37, 86.94, 86.72, 84.37, 85.22, 86.51, 83.3, 86.08, 86.3, 83.73, 84.15, 87.37, 84.58, 85.44, 84.58, 87.15, 82.01, 97.64]


In [20]:
scores = []
test_acc = []

for col in target:
    y = hkong[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier = RandomForestClassifier()

    cv_score = np.mean(cross_val_score(classifier, X_test, y_test, cv=10, scoring='roc_auc'))
    scores.append(cv_score)
    #print('CV score for class {} is {}'.format(col, cv_score))

    classifier.fit(X_train, y_train)
    accuracy_train = accuracy_score(y_train, classifier.predict(X_train))

    accuracy_test = accuracy_score(y_test, classifier.predict(X_test))
    #print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))

    #print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))
    test_acc.append(accuracy_test*100)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

print ('Mean Test Accuracy is {: .2f}%'.format(np.mean(test_acc)))
test_acc = map(lambda x:round(x,2), test_acc)
test_acc_RFC_1 =test_acc
print test_acc_RFC_1

Total CV score is 0.960134755156
Mean Test Accuracy is  94.15%
[92.51, 93.15, 92.72, 94.43, 88.87, 96.36, 95.72, 93.36, 96.36, 88.44, 95.29, 92.93, 97.43, 89.29, 91.86, 91.22, 94.65, 96.79, 94.0, 95.72, 97.86, 97.0, 95.5, 92.29, 97.22, 94.22, 94.0, 94.65, 91.65, 98.93]


In [21]:
scores = []
test_acc = []
for col in target:
    y = hkong[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier = GradientBoostingClassifier()

    cv_score = np.mean(cross_val_score(classifier, X_test, y_test, cv=10, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(col, cv_score))

    classifier.fit(X_train, y_train)
    accuracy_train = accuracy_score(y_train, classifier.predict(X_train))

    accuracy_test = accuracy_score(y_test, classifier.predict(X_test))
    #print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))

    #print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))
    test_acc.append(accuracy_test*100)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

print ('Mean Test Accuracy is {: .2f}%'.format(np.mean(test_acc)))
test_acc = map(lambda x:round(x,2), test_acc)
test_acc_GBC_1 =test_acc
print test_acc_GBC_1

CV score for class Target_0 is 1.0
CV score for class Target_1 is 0.997826086957
CV score for class Target_2 is 1.0
CV score for class Target_3 is 1.0
CV score for class Target_4 is 1.0
CV score for class Target_5 is 1.0
CV score for class Target_6 is 1.0
CV score for class Target_7 is 1.0
CV score for class Target_8 is 1.0
CV score for class Target_9 is 1.0
CV score for class Target_10 is 1.0
CV score for class Target_11 is 1.0
CV score for class Target_12 is 1.0
CV score for class Target_13 is 1.0
CV score for class Target_14 is 1.0
CV score for class Target_15 is 1.0
CV score for class Target_16 is 1.0
CV score for class Target_17 is 1.0
CV score for class Target_18 is 1.0
CV score for class Target_19 is 1.0
CV score for class Target_20 is 0.997727272727
CV score for class Target_21 is 1.0
CV score for class Target_22 is 1.0
CV score for class Target_23 is 1.0
CV score for class Target_24 is 0.997619047619
CV score for class Target_25 is 1.0
CV score for class Target_26 is 1.0
CV sc

In [22]:
testacc_df = pd.DataFrame([hong[:30], test_acc_LR_1, test_acc_SVC_1, test_acc_RFC_1, test_acc_GBC_1]).T
testacc_df.columns = ['Stock Components', 'Logistic Regression % (0.1) (1-)', 'SVM % (0.1) (1-)', 'Random Forest % (0.1) (1-)', 'Gradient Boosting % (0.1) (1-)']
testacc_df['Stock Components'] = map(lambda x:x.replace('.csv', ''), testacc_df['Stock Components'])
testacc_df

,Stock Components,Logistic Regression % (0.1) (1-),SVM % (0.1) (1-),Random Forest % (0.1) (1-),Gradient Boosting % (0.1) (1-)
0,AAC Technologies Holdings Inc. (2018.HK),89.94,86.08,92.51,100
1,AIA Group Limited (1299.HK),93.58,84.15,93.15,99.79
2,BOC Hong Kong (Holdings) Limited (2388.HK),93.36,80.3,92.72,100
3,China Life Insurance Company Limited (2628.HK),91.86,85.22,94.43,100
4,China Mengniu Dairy Company Limited (2319.HK),91.86,85.22,88.87,100
5,China Mobile Limited (0941.HK),94.22,85.01,96.36,100
6,China Petroleum & Chemical Corporation(0386.HK),91.65,86.94,95.72,100
7,China Resources Land Limited (1109.HK),92.72,84.58,93.36,100
8,China Resources Power Holdings Company Limited...,90.79,83.51,96.36,100
9,CITIC Limited (0267.HK),92.51,81.58,88.44,100


In [23]:
testacc_df.to_csv('E:/span1_0p1.csv', index=False)